<a href="https://colab.research.google.com/github/mgholamikn/AdaptPose/blob/main/AdaptPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AdaptPose: Cross-dataset Adaptation of 3D Human Pose Estimation by Learnable Motion Generator**

Here we provide the following experiments: 

1.   Cross-dataset evaluation on 3DHP: source H3.6M, target: 3DHP
2.   Cross-dataset evaluation on 3DPW: source H3.6M, target: 3DPW
3.   Cross-dataset trainin on 3DHP: source H3.6M, target: 3DHP
4.   Cross-dataset trainin on 3DPW: source H3.6M, target: 3DPW


# Install Dependencies

请用python3.8环境，以确保能找到对应pytorch

In [ ]:
# 临时补充
# %pip install pexpect
# %pip install psutil
%pip install PyQt5
# %pip install distutils

In [ ]:
%pip install certifi==2020.12.5
%pip install cffi==1.14.0
%pip install cycler==0.10.0
%pip install kiwisolver==1.3.1
%pip install matplotlib==3.1.3
# %pip install mkl-fft==1.3.0
# pip上只有很老的版本，需要conda安装
%conda install -c intel mkl_fft==1.3.0
%pip install mkl-random==1.1.1
%pip install mkl-service==2.3.0
%pip install nbconvert==5.6.1
%pip install numpy 
%pip install olefile==0.46
%pip install Pillow 
%pip install protobuf==3.15.6
%pip install pycparser 
%pip install pyparsing==2.4.7
%pip install python-dateutil==2.8.1
%pip install scipy==1.4.1
%pip install six 
%pip install tensorboardX==1.6
# %pip install torch
%conda install pytorch torchvision pytorch-cuda=11.6 -c pytorch -c nvidia
%pip install torchgeometry==0.1.2
# %pip install torchvision==0.2.2

## Pytorch安装

In [ ]:
import sys
import torch
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{torch.__version__[0:5:2]}"
])
%pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

# 下载数据

实验室服务器连不到google，需要手动下载后上传

In [ ]:
# #%cd /TO/AdaptPose/ 
# !ls
# %cd data
# ## download data
# !gdown --id 1_cJ2vG5wxRQUtWavz47XAwaMvU9pQB_X
# !gdown --id 1NX9dT1nKX-8t_eZR3vnfrdyOegG7pDBs
# !gdown --id 1v3-LDnjlHT8OJi40E7OKDBGY7dWM9P2n
# !gdown --id 12voJG5DBFy_2xT3lNdLYA4ZCTYjs7Iyv
# !gdown --id 1Y96GlQOEkc2Gx6V4FpvTwCyJjrUX3FYu
# !gdown --id 1vL53iJ1mWao3TKK3p52ZX_jdF5d5EBtl
# !gdown --id 1EbVV-nzrrQ2KdT_GG4JPwv1A7EOnFka1
# ## download pretrained models
# %cd ..
# %cd checkpoint/adaptpose/videopose/gt/3dhp/
# !gdown --id 11QJf-B5D1_aqsMR3v-9rrfht4W7ErC4k
# %cd ../../../../..
# %cd checkpoint/adaptpose/videopose/gt/3dpw/
# !gdown --id 1LUi1OQ7vWQ5KXJ9HlSvtveiOEacG1WoS
# %cd ../../../../..
# %cd checkpoint/adaptpose/videopose/gt/skii/
# !gdown --id 1b1_A7SuBdGaPNXzsx2qLOg0_RqIuk6Cx
# %cd ../../../../..
# %cd checkpoint/pretrain_baseline/videopose/gt/3dhp/
# !gdown --id 1SmSXrk_LcpsXBpBEsSp0zq9qIpb4aJBU
# %cd ../../../../..

# Experiments

## 1. 作者提供的3DHP迁移后最佳表现

In [ ]:
!python3  run_evaluate.py --posenet_name 'videopose' --keypoints gt --evaluate  'checkpoint/adaptpose/videopose/gt/3dhp/ckpt_best_dhp_p1.pth.tar' --dataset_target 3dhp --keypoints_target 'gt' --pad 13  --pretrain_path  'checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'

## 2. 对3DHP做迁移训练

In [1]:
epoch_num=20
# nohup python3 run_adaptpose.py --note poseaug --posenet_name 'videopose' --lr_p 1e-4 --checkpoint './checkpoint/adaptpose' --keypoints gt --keypoints_target gt --dataset_target '3dhp'  --pretrain_path './checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'  --pad 13 >>train.log 2>&1 &
# notebook不支持后台运行，copy上面这行手动跑
!python3 run_adaptpose.py --note poseaug --posenet_name 'videopose' --lr_p 1e-4 --checkpoint './checkpoint/adaptpose' --keypoints gt --keypoints_target gt --dataset_target '3dhp'  --pretrain_path './checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'  --pad 13 --epochs {epoch_num}


^C
Traceback (most recent call last):
  File "run_adaptpose.py", line 183, in <module>
    main(args)
  File "run_adaptpose.py", line 31, in main
    data_dict = data_preparation(args)
  File "/home/data/yyc/ap/function_adaptpose/data_preparation.py", line 75, in data_preparation
    'data', 'data_'+args.dataset_target + '_' + args.keypoints_target + '_train.npz'), dataset)
  File "/home/data/yyc/ap/utils/data_utils.py", line 41, in create_2d_data_target
    keypoints[subject][seq][cam_idx][ii,:,:] = normalize_screen_coordinates(keypoints[subject][seq][cam_idx][ii,joints,:2], w=2048, h=2048)
  File "/home/data/yyc/ap/common/camera.py", line 14, in normalize_screen_coordinates
    return X / w * 2 - [1, h / w]
KeyboardInterrupt


## 3. 在3DHP上测试未经迁移的模型

In [ ]:
!python3  run_evaluate.py --posenet_name 'videopose' --keypoints gt --evaluate 'checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar' --dataset_target 3dhp --keypoints_target 'gt' --pad 13

## 4. 在3DHP上测试复现训练的模型

In [ ]:
test_folder_name="trained3dhp2"
!python3  run_evaluate.py --posenet_name 'videopose' --keypoints gt --evaluate  'checkpoint/adaptpose/videopose/gt/{test_folder_name}/ckpt_best_dhp_p1.pth.tar' --dataset_target '3dhp' --keypoints_target 'gt' --pad 13